In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import matplotlib.pylab as plt
from scipy.interpolate import interp1d
from scipy.optimize import least_squares
import matplotlib.ticker as ticker  
from scipy.integrate import quad

import sys
sys.path.append('..')

from analytical_option_formulae.option_types.vanilla_option import VanillaOption
vanilla_option = VanillaOption()

#please adjust this before running, its either SPX or SPY
filename = 'SPX_options'

In [2]:
#implied volatility reporting

def implied_volatility(S: float, K: float, r: float, price: float, T: float, options_type: str) -> float:
    try:
        bs_model = lambda x: vanilla_option.black_scholes_model(S, K, r, x, T)
        if (options_type.lower() == 'call'):
            implied_vol = brentq(lambda x: price -
                                bs_model(x).calculate_call_price(),
                                1e-12, 10.0)
        elif (options_type.lower() == 'put'):
            implied_vol = brentq(lambda x: price -
                                bs_model(x).calculate_put_price(),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        implied_vol = np.nan

    return implied_vol


In [3]:
#Read SPX data
df = pd.read_csv(f'{filename}.csv')
df['mid'] = 0.5*(df['best_bid'] + df['best_offer'])
df['strike'] = df['strike_price']*0.001
df['payoff'] = df['cp_flag'].map(lambda x: 'call' if x == 'C' else 'put')
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['exdate'] = pd.to_datetime(df['exdate'], format='%Y%m%d')
df['days_to_expiry'] = (df['exdate'] - df['date']).dt.days
df['years_to_expiry'] = df['days_to_expiry']/365
#setup rates calculator
rates_df = pd.read_csv('zero_rates_20201201.csv')
rate_interpolate = interp1d(rates_df['days'], rates_df['rate'])
df['rates'] = rate_interpolate(df['days_to_expiry']) / 100.0 #make it in fractions so i dont forget

try:
    if filename.lower() == 'spy_options':
        S = 366.02
    elif filename.lower() == 'spx_options':
        S = 3662.45
    else:
        raise NameError('unknown input file')
except Exception as e:
    print(e)

#impl market volatility column
df['vols'] = df.apply(lambda x: implied_volatility(S, x['strike'], x['rates'], x['mid'], x['years_to_expiry'], x['payoff']),axis=1)
df.dropna(inplace=True)



In [4]:
#create market DF for each timestamp
days_to_expiry  = sorted(df['days_to_expiry'].unique())
summary_df = pd.DataFrame({'strike': df['strike'].unique() })
option_type = []
for days in days_to_expiry:
    day_df  = df[df['days_to_expiry'] == days]
    call_df = day_df[day_df['payoff'] == 'call']
    put_df  = day_df[day_df['payoff'] == 'put']
    strikes = sorted(day_df['strike'].unique())
    impliedvols = []
    for K in strikes:    
        if K > S:
            impliedvols.append(call_df[call_df['strike'] == K]['vols'].values[0])
            option_type.append('call')
        else:
            impliedvols.append(put_df[put_df['strike'] == K]['vols'].values[0])
            option_type.append('put')

    day_market_df = pd.DataFrame({'strike': strikes, days : impliedvols})
    summary_df = pd.merge(summary_df, day_market_df, how="outer", on='strike')

summary_df['option_type'] = summary_df.apply(lambda x: 'call' if x['strike'] > S else 'put', axis=1)
summary_df = summary_df.sort_values(by=['strike'])
summary_df = summary_df.reset_index()
summary_df = summary_df.drop(columns=['index'])
summary_df.to_csv(f'{filename}_vol_summary.csv')

In [5]:
#get atm implied vols, this is an approximation but good enough
atm_vols = {}
for i,days in enumerate(days_to_expiry):
    adjusted_summary_df = summary_df.iloc[:, [0,1+i,-1]].copy()
    adjusted_summary_df.dropna(inplace=True)    
    atm_vols[days] = np.interp(S,adjusted_summary_df.iloc[:, 0],adjusted_summary_df.iloc[:, 1])
atm_vols

{17: 0.17448532472572187, 45: 0.1849096526276905, 80: 0.19374721854522414}

In [6]:
start_date  = pd.to_datetime('2020-12-01')
end_date    = pd.to_datetime('2021-01-15')
day_diff = (end_date-start_date).days
T = day_diff/365.0
rate = rate_interpolate(day_diff)/100.0
day_diff

45

Expected Black Scholes payoff is defined as
$$
E[V_T]= {S_0}^\frac{1}{3}e^{\frac{1}{3}(r-\frac{\sigma^2}{2})T}e^{\frac{1}{2}\frac{\sigma^2}{9}T} + 1.5(log{S_0} + (r-\frac{\sigma^2}{2})T) + 10
$$

Therefore, the price is
$$
V_0 = e^{-rT}E[V_T]
$$


In [7]:
#black scholes payoff
sigma = np.linspace(0,0.3,100)

def bs_price(S,rate,sigma,T):
    return np.exp(-rate*T) *((np.power(S,1.0/3.0) * np.exp((rate - 0.5 * sigma ** 2) * T * (1.0/3.0)) * np.exp(0.5 * (1.0/9.0) * T * sigma**2) \
                + 1.5* (np.log(S) + (rate - 0.5 * sigma ** 2) * T)  \
                + 10))


In [8]:
print(f'S = {S}, rate = {rate} ,sigma = {atm_vols[day_diff]}, T = {T}')

S = 3662.45, rate = 0.0020510755555555554 ,sigma = 0.1849096526276905, T = 0.1232876712328767


In [9]:
bs_price(S,rate,atm_vols[day_diff],T)

37.70489747924678

Expected Bachelier payoff defined as

$$
E[V_T] = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} (S_0 + \sigma S_0 x)^\frac{1}{3} e^\frac{-x^2}{2}\,dx +
\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} 1.5log(S_0 + \sigma S_0 x) e^\frac{-x^2}{2}\,dx
+10
$$

Note  $$S_T = S_0 + \sigma S_0W_T$$

Therefore, the price is
$$
V_0 = e^{-rT}E[V_T]
$$


In [10]:
def integrand_1(x,S,sigma):
    return (1/np.sqrt(2*np.pi)) * np.power((S + sigma * S * x), 1.0/3.0) * np.exp(-np.power(x,2)/2)
def integrand_2(x,S,sigma):
    return (1/np.sqrt(2*np.pi)) * 1.5* np.log(S + sigma * S * x) * np.exp(-np.power(x,2)/2)
sigma = atm_vols[day_diff]
I_1 = quad(lambda x: integrand_1(x,S,sigma), 1, 1e10)
print('The integral is: %.9f' % I_1[0])
I_2 = quad(lambda x: integrand_2(x,S,sigma), 1, 1e10)
print('The integral is: %.9f' % I_2[0])

expected_payoff = I_1[0] + I_2[0] + 10
print(f'The expected payoff is: {expected_payoff}')


The integral is: 0.000000000
The integral is: 0.000000000
The expected payoff is: 10.0


# Barone-Adesi & Whaley Approximation for American Call #

The value of an American Call Option can be approximated to be

$$
C(S,T) = c(S,T) + A_2\left(\frac{S}{S^*}\right)^{q_2}   ; S < S^* \\
C(S,T) = S- X  ; S \geq S^*
$$

Where 
* $S$ is underlying commodity price (e.g. stock, etc)
* $S^*$ is critical commodity price
* $T$ is time to expiration
* $X$ is the strike price

We can see the parameters that we need  are
* $A_2$
* $S^*$
* $q_2$

$$
K(T)= 1-e^{-rT}\\
M = \frac{2r}{\sigma^2}\\
N = \frac{2b}{\sigma^2}\\
q_2 = \frac{\left[-(N - 1) + \left(\sqrt{(N - 1)^2 + \frac{4M}{K}}\right)\right]}{2}\\
A_2 = \frac{S^*}{q_2} (1 - e^{(b-r)T}N[d_1(S^*)]) \\
d_1 = \frac{ln(\frac{S}{X}) + (b + \frac{\sigma^2}{2})T}{\sigma \sqrt{T}} \:  \text{(seem familiar?)} \\
S^* - X = c(S^*,T) + \frac{S^*}{q_2} (1 - e^{(b-r)T}N[d_1(S^*)])
$$

We could further simplify the formula : First, the short-term interest rate, r, and the cost of carrying the commodity, b, are assumed to be constant, proportional rates.For a non-dividend-paying stock, the cost of carry is equal to the riskless rate of interest (i.e., b = r).


Note that the the authors mention that $S^*$ must be determined iteratively. Or using an approximator formula


# Barone-Adesi & Whaley Approximation for American Put #

The value of an American Call Option can be approximated to be

$$
P(S,T) = p(S,T) + A_a\left(\frac{S}{S^{**}}\right)^{q_2}   ; S > S^{**} \\
P(S,T) = X - S  ; S \leq S^{**}
$$

Where 
* $S$ is underlying commodity price (e.g. stock, etc)
* $S^*$ is critical commodity price
* $T$ is time to expiration
* $X$ is the strike price

We can see the parameters that we need  are
* $A_2$
* $S^*$
* $q_2$

$$
K(T)= 1-e^{-rT}\\
M = \frac{2r}{\sigma^2}\\
N = \frac{2b}{\sigma^2}\\
q_1 = \frac{\left[-(N - 1) - \left(\sqrt{(N - 1)^2 + \frac{4M}{K}}\right)\right]}{2}\\
A_1 = -\frac{S^{**}}{q_1} (1 - e^{(b-r)T}N[-d_1(S^{**})]) \\
X - S^{**}= p(S^{**},T)- (1 - e^{(b-r)T}N[-d_1(S^{**})]) \\
$$

We could further simplify the formula : First, the short-term interest rate, r, and the cost of carrying the commodity, b, are assumed to be constant, proportional rates.For a non-dividend-paying stock, the cost of carry is equal to the riskless rate of interest (i.e., b = r).


Note that the the authors mention that $S^{**}$ must be determined iteratively. Or using an approximator formula
